<a href="https://colab.research.google.com/github/fabiofaggian/near_of_earth_objects_NASA/blob/main/Asteroides_reprocess_hist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import

import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def openjson(path, filename):
  with open(path + filename, 'r') as fp:
    data = json.load(fp)
    return data

def dates(start_date, end_date):
    date_sun = pd.DataFrame(pd.date_range(f"{start_date}", f"{end_date}", freq="W-SUN"))
    date_sat = pd.DataFrame(pd.date_range(f"{start_date}", f"{end_date}", freq="W-SAT"))

    dfdates_sun = []
    dfdates_sat = []

    for r in range(len(date_sun)):
      dfdates_sun.append(date_sun.loc[r,0].strftime("%Y-%m-%d"))
      dfdates_sat.append(date_sat.loc[r,0].strftime("%Y-%m-%d"))
    return (dfdates_sun, dfdates_sat)

In [ ]:
path = '/content/drive/MyDrive/Testes/base_dados/raw_historico'

START = '2000-01-02'   #Inserir o primeiro domingo
END = '2022-08-27'     #Inserir o primeiro sabado
#2009-11-15_2009-11-21  PROBLEM

#INGESTÃO de dados do passado

sunday = dates(START, END)[0]
saturday = dates(START, END)[1]

#Quantidade de semanas que será processada
print(len(dates(START, END)[0]))

1182


In [ ]:
w =len(sunday)
df_raw = pd.DataFrame()

for i in range(w):
    data = pd.DataFrame(openjson(path,f'/{sunday[i]}_{saturday[i]}.json'))
    df_raw = df_raw.append(data)


df_raw = df_raw.drop(['next','prev','self']).drop(columns=['links','element_count'])
df_raw = df_raw.reset_index().sort_values(by=['index'])

#Tratamento 

# Tratar os dados

others = ['close_approach_date','close_approach_date_full','epoch_date_close_approach','orbiting_body']
vel = ['kilometers_per_hour','kilometers_per_second','miles_per_hour']
dist = ['astronomical','kilometers','lunar','miles']
diam = ['feet','meters','kilometers','miles']

df_final = pd.DataFrame()
ze = len(df_raw)

for n in range(len(df_raw)):
  df = pd.DataFrame(df_raw['near_earth_objects'][n])
  for i in range(len(df)):
    df.loc[i,'links'] = df.loc[i,'links']['self']
    for j in others:
      df.loc[i,j] = df.loc[i].close_approach_data[0][j]
    for k in vel:
      df.loc[i,f'vel_{k}'] = df.loc[i].close_approach_data[0]['relative_velocity'][k]
    for l in dist:
      df.loc[i,f'miss_dist_{l}'] = df.loc[i].close_approach_data[0]['miss_distance'][l]
    for m in diam:
      try:
        df.loc[i,f'diam_max_{m}'] = df.loc[i]['estimated_diameter'][m]['estimated_diameter_max']
        df.loc[i,f'diam_min_{m}'] = df.loc[i]['estimated_diameter'][m]['estimated_diameter_min']
      except:
        df.loc[i,f'diam_max_{m}'] = 0
        df.loc[i,f'diam_min_{m}'] = 0
  df_final = df_final.append(df.drop(['estimated_diameter','close_approach_data'], axis=1))
  #df = df.drop(['estimated_diameter','close_approach_data'], axis=1)
  print(end="\r%6.2f %%" % (n/(ze-1)*100))

100.00 %

In [ ]:
df_final = df_final.reset_index()
df_final = df_final.drop(columns='index')

In [ ]:
df_final.to_csv('/content/drive/MyDrive/Testes/base_dados/refined_historico/refined_csv')